In [1]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from pathlib import Path
import numpy as np
import PIL
import PIL.Image
import json

#change working directory to root
ROOT_DIR = os.getcwd()
while os.path.basename(ROOT_DIR) != 'VisIrNet':
    ROOT_DIR = os.path.abspath(os.path.join(ROOT_DIR,'..'))
sys.path.insert(0,ROOT_DIR)
os.chdir(ROOT_DIR)

ROOT_DIR = Path(ROOT_DIR)

print(tf.__version__)
devices = tf.config.list_physical_devices('GPU')
print("len(devices): ", len(devices))
print(f"available GPUs: {devices}");


2.14.0
len(devices):  0
available GPUs: []


**Dataloaders**

In [2]:
import data_setup

# try to import the dataset
dataset="SkyData"
BATCH_SIZE = 2
SHUFFLE_BUFFER_SIZE = 100

train_dataloader,test_dataloader = data_setup.create_dataloaders(dataset=dataset, 
                                                                BATCH_SIZE=BATCH_SIZE,
                                                                SHUFFLE_BUFFER_SIZE=100
                                                                )
#

[INFO] loading train dataset
[INFO] train _dataset:  27700
[INFO] loading val dataset
[INFO] val _dataset:  7990


## **Model**

In [3]:
import model_setup
import Utils
rgb_inputs_shape = (192,192,3)
ir_inputs_shape =  (128,128,3)
output_channels_per_block = 3
regression_input_shape = (*rgb_inputs_shape[:2],output_channels_per_block*2)
regression_output_shape = 8


In [4]:
featureEmbeddingBackBone = model_setup.getFeatureEmbeddingBackBone(rgb_inputs_shape=rgb_inputs_shape,
                                                        ir_inputs_shape=ir_inputs_shape,
                                                        output_channels_per_block=output_channels_per_block
                                                        )

regressionHead= model_setup.getRegressionHead(input_shape=regression_input_shape,
                                                output_size=regression_output_shape
                                                )


**Visualize and save model structures**

In [5]:
# # visualize and save models

# Utils.plot_and_save_model_structure(featureEmbeddingBackBone,
#                                             save_path="resources/",
#                                             save_as=f"featureEmbeddingBackBone")
# Utils.plot_and_save_model_structure(regressionHead,
#                                             save_path="resources/",
#                                             save_as=f"regressionHead")

## **Training**


In [6]:
import engine 

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,
                                                                decay_steps=10000,
                                                                decay_rate=0.96,
                                                                staircase=True)
NUM_EPOCHS = 2

# Setup optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Start the timer
from timeit import default_timer as timer
start_time = timer()
# Train model 

model_results = engine.train_first_stage(model=featureEmbeddingBackBone,
                                                train_dataloader=train_dataloader,
                                                test_dataloader=test_dataloader,
                                                optimizer=optimizer,
                                                epochs=NUM_EPOCHS,
                                                save_path="models",
                                                save_as=f"featureEmbeddingBackBone",
                                                save_frequency=1,
                                                save_hard_frequency=50
                                                )
# End the timer and print out how long it took
end_time = timer()
print(f"Total training time : {end_time-start_time:.3f} seconds")

/home/nalain/miniconda3/envs/VisIrNet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] TrainingfeatureEmbeddingBackbone for 2 epochs
[INFO] Epoch 1/2


8it [00:10,  1.36s/it]


[train_loss] : fir_frgb :41588.957|fir_Iir :99035.47|frgb_Irgb :23541.602|fir_Irgb :119831.8|frgb_Iir :41588.957|ir_Irgb :25170.664|total_loss :185061.38


/home/nalain/miniconda3/envs/VisIrNet/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[test_loss] : fir_frgb :60999.83|fir_Iir :74608.17|frgb_Irgb :64984.05|fir_Irgb :48580.875|frgb_Iir :60999.83|ir_Irgb :26470.793|total_loss :174639.38
[INFO] Saving model at models
[INFO] Epoch 2/2


8it [00:09,  1.24s/it]


[train_loss] : fir_frgb :19682.773|fir_Iir :35949.527|frgb_Irgb :12828.844|fir_Irgb :32659.049|frgb_Iir :19682.773|ir_Irgb :20011.578|total_loss :65206.62
[test_loss] : fir_frgb :14670.064|fir_Iir :11033.993|frgb_Irgb :2186.2651|fir_Irgb :38423.285|frgb_Iir :14670.064|ir_Irgb :20982.21|total_loss :55290.65
[INFO] Saving model at models
[INFO] Saving logs at logs
Total training time : 29.699 seconds
